In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
!ls

EDA and metrics.ipynb model                 server.py
README.md             mvnw                  src
data                  mvnw.cmd
dataset               pom.xml


In [169]:
raw = pd.read_csv('data/host.csv', names=['host'])

In [172]:
raw.drop_duplicates()

,host
0,api.youla.io
1,favicon.yandex.net
2,w-74721.fp.kaspersky-labs.com
3,questtime.net
4,passport-authproxy.taxi.yandex.net
...,...
999991,m39.hefxosi.com
999993,m42.ytoxcrj.com
999995,m5.fupppzz.net
999997,m5.ehgeqxn.me


In [90]:
def df_process(df):
    df['len'] = df['host'].map(lambda x: len(x))
    df['dots'] = df['host'].map(lambda x: x.count('.'))
    df['digits'] = df['host'].map(lambda x: sum([x.count(str(c)) for c in range(10)]))
    df['dash'] = df['host'].map(lambda x: x.count('-'))    
    return df

In [13]:
raw = df_process(raw)

In [14]:
raw.describe()

,len,dots,digits,dash
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,22.485400,2.297148,2.293751,0.712215
std,11.133069,1.093478,4.588589,1.218499
min,4.000000,1.000000,0.000000,0.000000
25%,15.000000,2.000000,0.000000,0.000000
50%,19.000000,2.000000,1.000000,0.000000
75%,27.000000,3.000000,2.000000,1.000000
max,84.000000,32.000000,41.000000,11.000000


In [292]:
dataset = pd.read_csv('dataset/data.csv', index_col=0)

In [297]:
dataset.index = range(len(dataset))

In [298]:
dataset

,host,y
0,pcvrc.com,1
1,m25.dlfynky.net,0
2,coconutgrovekeywest.com,1
3,lisacerasoli.com,1
4,cal25.com,1
...,...,...
345657,sunmines.com.tw,1
345658,doksan.com,1
345659,9f398ace-e1c4-442a-9cb3-345e01db1c26.mitdmp.wh...,0
345660,m40.cnoyucn.com,0


In [299]:
dataset = df_process(dataset)

In [300]:
dataset.describe()

,y,len,dots,digits,dash
count,345662.000000,345662.000000,345662.000000,345662.000000,345662.000000
mean,0.500000,22.523734,1.836505,3.376003,0.465281
std,0.500001,14.686289,1.522946,6.592448,1.155138
min,0.000000,2.000000,0.000000,0.000000,0.000000
25%,0.000000,14.000000,1.000000,0.000000,0.000000
50%,0.500000,17.000000,2.000000,0.000000,0.000000
75%,1.000000,25.000000,2.000000,2.000000,0.000000
max,1.000000,84.000000,32.000000,40.000000,11.000000


In [301]:
user = dataset[dataset['y'] == 1]
tech = dataset[dataset['y'] == 0]

In [302]:
user.describe()

,y,len,dots,digits,dash
count,172831.0,172831.000000,172831.000000,172831.000000,172831.000000
mean,1.0,16.245130,1.234674,0.066730,0.109870
std,0.0,5.089179,0.494328,0.404235,0.371298
min,1.0,2.000000,0.000000,0.000000,0.000000
25%,1.0,13.000000,1.000000,0.000000,0.000000
50%,1.0,16.000000,1.000000,0.000000,0.000000
75%,1.0,19.000000,1.000000,0.000000,0.000000
max,1.0,67.000000,5.000000,13.000000,10.000000


In [162]:
tech.describe()

,y,len,dots,digits,dash
count,172831.0,172831.000000,172831.000000,172831.000000,172831.000000
mean,0.0,28.802339,2.438336,6.685276,0.820692
std,0.0,18.073004,1.915719,8.053253,1.509371
min,0.0,4.000000,1.000000,0.000000,0.000000
25%,0.0,15.000000,2.000000,1.000000,0.000000
50%,0.0,22.000000,2.000000,2.000000,0.000000
75%,0.0,38.000000,3.000000,12.000000,1.000000
max,0.0,84.000000,32.000000,40.000000,11.000000


In [305]:
print(f'соотношение классов user/tech: {len(user) / len(tech)}')

соотношение классов user/tech: 1.0


In [306]:
from sklearn.utils import shuffle
dataset = shuffle(dataset)

In [313]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

In [310]:
kf = KFold(n_splits=5, shuffle=True)

In [311]:
from model.Model import Model

In [312]:
X = dataset['host'].values
y = dataset['y'].values

In [291]:
metrics = {
    'accuracy': [],
    'roc_auc': [],
    'f1_score': [],
}

from tqdm import tqdm
for train_index, test_index in tqdm(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = Model(depth=8)
    model.fit(X_train, y_train, verbose=False)
    y_pred = model.predict(X_test)
    metrics['accuracy'].append(accuracy_score(y_test, y_pred))
    metrics['roc_auc'].append(roc_auc_score(y_test, y_pred))
    metrics['f1_score'].append(f1_score(y_test, y_pred))

0it [00:00, ?it/s]

Learning rate set to 0.113647
0:	learn: 0.5363840	total: 56.9ms	remaining: 56.9s
1:	learn: 0.4411006	total: 97.6ms	remaining: 48.7s
2:	learn: 0.3746483	total: 138ms	remaining: 46s
3:	learn: 0.3323217	total: 182ms	remaining: 45.2s
4:	learn: 0.3011519	total: 230ms	remaining: 45.8s
5:	learn: 0.2817009	total: 276ms	remaining: 45.7s
6:	learn: 0.2648239	total: 319ms	remaining: 45.2s
7:	learn: 0.2523484	total: 362ms	remaining: 44.9s
8:	learn: 0.2440400	total: 411ms	remaining: 45.3s
9:	learn: 0.2357817	total: 459ms	remaining: 45.4s
10:	learn: 0.2286515	total: 497ms	remaining: 44.7s
11:	learn: 0.2233234	total: 539ms	remaining: 44.4s
12:	learn: 0.2182893	total: 578ms	remaining: 43.9s
13:	learn: 0.2137817	total: 615ms	remaining: 43.3s
14:	learn: 0.2102563	total: 657ms	remaining: 43.2s
15:	learn: 0.2074033	total: 696ms	remaining: 42.8s
16:	learn: 0.2039367	total: 735ms	remaining: 42.5s
17:	learn: 0.2013062	total: 778ms	remaining: 42.5s
18:	learn: 0.1986106	total: 822ms	remaining: 42.4s
19:	learn: 

161:	learn: 0.1404146	total: 7.82s	remaining: 40.5s
162:	learn: 0.1402869	total: 7.86s	remaining: 40.4s
163:	learn: 0.1400795	total: 7.9s	remaining: 40.3s
164:	learn: 0.1400089	total: 7.94s	remaining: 40.2s
165:	learn: 0.1398750	total: 7.99s	remaining: 40.1s
166:	learn: 0.1396703	total: 8.04s	remaining: 40.1s
167:	learn: 0.1395715	total: 8.09s	remaining: 40.1s
168:	learn: 0.1393734	total: 8.15s	remaining: 40.1s
169:	learn: 0.1392195	total: 8.2s	remaining: 40s
170:	learn: 0.1390512	total: 8.25s	remaining: 40s
171:	learn: 0.1389938	total: 8.29s	remaining: 39.9s
172:	learn: 0.1389308	total: 8.33s	remaining: 39.8s
173:	learn: 0.1388359	total: 8.37s	remaining: 39.7s
174:	learn: 0.1387490	total: 8.41s	remaining: 39.7s
175:	learn: 0.1386143	total: 8.45s	remaining: 39.6s
176:	learn: 0.1385057	total: 8.5s	remaining: 39.5s
177:	learn: 0.1384237	total: 8.54s	remaining: 39.4s
178:	learn: 0.1383892	total: 8.59s	remaining: 39.4s
179:	learn: 0.1382456	total: 8.63s	remaining: 39.3s
180:	learn: 0.13809

320:	learn: 0.1248109	total: 15.7s	remaining: 33.3s
321:	learn: 0.1248011	total: 15.8s	remaining: 33.2s
322:	learn: 0.1247568	total: 15.8s	remaining: 33.1s
323:	learn: 0.1247427	total: 15.8s	remaining: 33.1s
324:	learn: 0.1246799	total: 15.9s	remaining: 33s
325:	learn: 0.1246191	total: 15.9s	remaining: 32.9s
326:	learn: 0.1245354	total: 16s	remaining: 32.9s
327:	learn: 0.1245084	total: 16s	remaining: 32.8s
328:	learn: 0.1244291	total: 16s	remaining: 32.7s
329:	learn: 0.1243205	total: 16.1s	remaining: 32.7s
330:	learn: 0.1242129	total: 16.1s	remaining: 32.6s
331:	learn: 0.1241766	total: 16.2s	remaining: 32.5s
332:	learn: 0.1241013	total: 16.2s	remaining: 32.5s
333:	learn: 0.1239759	total: 16.3s	remaining: 32.4s
334:	learn: 0.1238889	total: 16.3s	remaining: 32.4s
335:	learn: 0.1238243	total: 16.3s	remaining: 32.3s
336:	learn: 0.1237217	total: 16.4s	remaining: 32.2s
337:	learn: 0.1236702	total: 16.4s	remaining: 32.2s
338:	learn: 0.1236033	total: 16.5s	remaining: 32.1s
339:	learn: 0.123520

481:	learn: 0.1156529	total: 23.6s	remaining: 25.4s
482:	learn: 0.1156460	total: 23.7s	remaining: 25.3s
483:	learn: 0.1156147	total: 23.7s	remaining: 25.3s
484:	learn: 0.1155983	total: 23.8s	remaining: 25.2s
485:	learn: 0.1155354	total: 23.8s	remaining: 25.2s
486:	learn: 0.1155187	total: 23.9s	remaining: 25.1s
487:	learn: 0.1155079	total: 23.9s	remaining: 25.1s
488:	learn: 0.1155068	total: 23.9s	remaining: 25s
489:	learn: 0.1154367	total: 24s	remaining: 25s
490:	learn: 0.1154148	total: 24.1s	remaining: 24.9s
491:	learn: 0.1154086	total: 24.1s	remaining: 24.9s
492:	learn: 0.1153651	total: 24.2s	remaining: 24.9s
493:	learn: 0.1153484	total: 24.3s	remaining: 24.9s
494:	learn: 0.1153133	total: 24.3s	remaining: 24.8s
495:	learn: 0.1152660	total: 24.4s	remaining: 24.8s
496:	learn: 0.1151973	total: 24.8s	remaining: 25.1s
497:	learn: 0.1151894	total: 24.9s	remaining: 25.1s
498:	learn: 0.1151819	total: 25s	remaining: 25.1s
499:	learn: 0.1151566	total: 25s	remaining: 25s
500:	learn: 0.1151090	to

...

807:	learn: 0.1023527	total: 39.6s	remaining: 9.41s
808:	learn: 0.1023012	total: 39.7s	remaining: 9.37s
809:	learn: 0.1022862	total: 39.7s	remaining: 9.32s
810:	learn: 0.1022279	total: 39.8s	remaining: 9.27s
811:	learn: 0.1021729	total: 39.8s	remaining: 9.22s
812:	learn: 0.1021373	total: 39.9s	remaining: 9.17s
813:	learn: 0.1020974	total: 39.9s	remaining: 9.12s
814:	learn: 0.1020679	total: 39.9s	remaining: 9.07s
815:	learn: 0.1020533	total: 40s	remaining: 9.02s
816:	learn: 0.1020388	total: 40s	remaining: 8.97s
817:	learn: 0.1020030	total: 40.1s	remaining: 8.92s
818:	learn: 0.1019709	total: 40.1s	remaining: 8.87s
819:	learn: 0.1019424	total: 40.2s	remaining: 8.82s
820:	learn: 0.1018798	total: 40.2s	remaining: 8.78s
821:	learn: 0.1018227	total: 40.3s	remaining: 8.73s
822:	learn: 0.1017930	total: 40.4s	remaining: 8.68s
823:	learn: 0.1017465	total: 40.4s	remaining: 8.63s
824:	learn: 0.1017011	total: 40.5s	remaining: 8.58s
825:	learn: 0.1016874	total: 40.5s	remaining: 8.53s
826:	learn: 0.10

970:	learn: 0.0967932	total: 47.7s	remaining: 1.43s
971:	learn: 0.0967724	total: 47.8s	remaining: 1.38s
972:	learn: 0.0967417	total: 47.8s	remaining: 1.33s
973:	learn: 0.0967221	total: 47.9s	remaining: 1.28s
974:	learn: 0.0966601	total: 47.9s	remaining: 1.23s
975:	learn: 0.0966231	total: 48s	remaining: 1.18s
976:	learn: 0.0965849	total: 48s	remaining: 1.13s
977:	learn: 0.0965394	total: 48.1s	remaining: 1.08s
978:	learn: 0.0965099	total: 48.1s	remaining: 1.03s
979:	learn: 0.0964697	total: 48.2s	remaining: 983ms
980:	learn: 0.0964447	total: 48.2s	remaining: 934ms
981:	learn: 0.0964042	total: 48.3s	remaining: 884ms
982:	learn: 0.0963373	total: 48.3s	remaining: 835ms
983:	learn: 0.0963233	total: 48.3s	remaining: 786ms
984:	learn: 0.0962901	total: 48.4s	remaining: 737ms
985:	learn: 0.0962438	total: 48.4s	remaining: 688ms
986:	learn: 0.0962069	total: 48.5s	remaining: 638ms
987:	learn: 0.0961645	total: 48.5s	remaining: 589ms
988:	learn: 0.0961222	total: 48.6s	remaining: 540ms
989:	learn: 0.09

0it [00:53, ?it/s]

999:	learn: 0.0956649	total: 49.1s	remaining: 0us


CatBoostError: (No such file or directory) util/system/file.cpp:848: can't open "../saved/" with mode WrOnly|CreateAlways|Seq (0x00000034)

In [278]:
for k, v in metrics.items():
    print(f'{k}: {np.round(np.mean(v), 4)}')

accuracy: 0.9502
roc_auc: 0.9502
precision: 0.9411
recall: 0.9599


In [283]:
print(str(round(100 * len(dataset[dataset['y'] == 1]) / len(dataset), 2)) + '%')

50.0%


In [208]:
from model.Model import Model

In [209]:
model = Model()

In [211]:
model.fit(dataset['host'], y)

Learning rate set to 0.129702
0:	learn: 0.5305536	total: 188ms	remaining: 3m 7s
1:	learn: 0.4287488	total: 316ms	remaining: 2m 37s
2:	learn: 0.3692330	total: 437ms	remaining: 2m 25s
3:	learn: 0.3279142	total: 555ms	remaining: 2m 18s
4:	learn: 0.3015253	total: 657ms	remaining: 2m 10s
5:	learn: 0.2813459	total: 779ms	remaining: 2m 9s
6:	learn: 0.2685742	total: 900ms	remaining: 2m 7s
7:	learn: 0.2564830	total: 1.01s	remaining: 2m 5s
8:	learn: 0.2488897	total: 1.13s	remaining: 2m 4s
9:	learn: 0.2422002	total: 1.25s	remaining: 2m 3s
10:	learn: 0.2363815	total: 1.36s	remaining: 2m 2s
11:	learn: 0.2329778	total: 1.47s	remaining: 2m
12:	learn: 0.2292937	total: 1.59s	remaining: 2m
13:	learn: 0.2261749	total: 1.73s	remaining: 2m 2s
14:	learn: 0.2236552	total: 1.83s	remaining: 2m
15:	learn: 0.2209117	total: 1.93s	remaining: 1m 58s
16:	learn: 0.2192877	total: 2.03s	remaining: 1m 57s
17:	learn: 0.2163612	total: 2.15s	remaining: 1m 57s
18:	learn: 0.2144905	total: 2.25s	remaining: 1m 56s
19:	learn: 0

158:	learn: 0.1606709	total: 20s	remaining: 1m 45s
159:	learn: 0.1605409	total: 20.2s	remaining: 1m 45s
160:	learn: 0.1604491	total: 20.3s	remaining: 1m 45s
161:	learn: 0.1602881	total: 20.5s	remaining: 1m 45s
162:	learn: 0.1601331	total: 20.6s	remaining: 1m 45s
163:	learn: 0.1599750	total: 20.8s	remaining: 1m 45s
164:	learn: 0.1598173	total: 20.9s	remaining: 1m 45s
165:	learn: 0.1596446	total: 21.1s	remaining: 1m 45s
166:	learn: 0.1594911	total: 21.4s	remaining: 1m 46s
167:	learn: 0.1592837	total: 21.6s	remaining: 1m 46s
168:	learn: 0.1590829	total: 21.7s	remaining: 1m 46s
169:	learn: 0.1590484	total: 21.8s	remaining: 1m 46s
170:	learn: 0.1589318	total: 22s	remaining: 1m 46s
171:	learn: 0.1587915	total: 22.2s	remaining: 1m 46s
172:	learn: 0.1586375	total: 22.3s	remaining: 1m 46s
173:	learn: 0.1585358	total: 22.5s	remaining: 1m 46s
174:	learn: 0.1583254	total: 22.6s	remaining: 1m 46s
175:	learn: 0.1580538	total: 22.7s	remaining: 1m 46s
176:	learn: 0.1579376	total: 23.1s	remaining: 1m 4

315:	learn: 0.1406697	total: 44.7s	remaining: 1m 36s
316:	learn: 0.1405710	total: 44.8s	remaining: 1m 36s
317:	learn: 0.1404452	total: 44.9s	remaining: 1m 36s
318:	learn: 0.1403471	total: 45s	remaining: 1m 36s
319:	learn: 0.1402282	total: 45.2s	remaining: 1m 35s
320:	learn: 0.1400848	total: 45.3s	remaining: 1m 35s
321:	learn: 0.1400181	total: 45.4s	remaining: 1m 35s
322:	learn: 0.1398538	total: 45.5s	remaining: 1m 35s
323:	learn: 0.1398159	total: 45.6s	remaining: 1m 35s
324:	learn: 0.1397053	total: 45.8s	remaining: 1m 35s
325:	learn: 0.1396047	total: 45.9s	remaining: 1m 34s
326:	learn: 0.1395253	total: 46s	remaining: 1m 34s
327:	learn: 0.1394571	total: 46.1s	remaining: 1m 34s
328:	learn: 0.1393823	total: 46.2s	remaining: 1m 34s
329:	learn: 0.1393012	total: 46.3s	remaining: 1m 34s
330:	learn: 0.1391776	total: 46.4s	remaining: 1m 33s
331:	learn: 0.1390822	total: 46.6s	remaining: 1m 33s
332:	learn: 0.1390143	total: 46.7s	remaining: 1m 33s
333:	learn: 0.1389400	total: 46.8s	remaining: 1m 3

471:	learn: 0.1282986	total: 1m 10s	remaining: 1m 19s
472:	learn: 0.1282038	total: 1m 10s	remaining: 1m 19s
473:	learn: 0.1281012	total: 1m 11s	remaining: 1m 19s
474:	learn: 0.1280661	total: 1m 11s	remaining: 1m 18s
475:	learn: 0.1279938	total: 1m 11s	remaining: 1m 18s
476:	learn: 0.1279445	total: 1m 11s	remaining: 1m 18s
477:	learn: 0.1278546	total: 1m 12s	remaining: 1m 18s
478:	learn: 0.1278052	total: 1m 12s	remaining: 1m 18s
479:	learn: 0.1277483	total: 1m 12s	remaining: 1m 18s
480:	learn: 0.1276658	total: 1m 12s	remaining: 1m 18s
481:	learn: 0.1276206	total: 1m 12s	remaining: 1m 18s
482:	learn: 0.1275474	total: 1m 13s	remaining: 1m 18s
483:	learn: 0.1274064	total: 1m 13s	remaining: 1m 18s
484:	learn: 0.1273101	total: 1m 13s	remaining: 1m 18s
485:	learn: 0.1271722	total: 1m 13s	remaining: 1m 17s
486:	learn: 0.1270928	total: 1m 13s	remaining: 1m 17s
487:	learn: 0.1270899	total: 1m 14s	remaining: 1m 17s
488:	learn: 0.1270629	total: 1m 14s	remaining: 1m 17s
489:	learn: 0.1270317	total:

625:	learn: 0.1194498	total: 1m 39s	remaining: 59.2s
626:	learn: 0.1194022	total: 1m 39s	remaining: 59s
627:	learn: 0.1193917	total: 1m 39s	remaining: 58.8s
628:	learn: 0.1192929	total: 1m 39s	remaining: 58.7s
629:	learn: 0.1192153	total: 1m 39s	remaining: 58.5s
630:	learn: 0.1191749	total: 1m 39s	remaining: 58.3s
631:	learn: 0.1191681	total: 1m 39s	remaining: 58.2s
632:	learn: 0.1191177	total: 1m 40s	remaining: 58s
633:	learn: 0.1190516	total: 1m 40s	remaining: 57.9s
634:	learn: 0.1189718	total: 1m 40s	remaining: 57.7s
635:	learn: 0.1188845	total: 1m 40s	remaining: 57.6s
636:	learn: 0.1188333	total: 1m 40s	remaining: 57.4s
637:	learn: 0.1187595	total: 1m 40s	remaining: 57.2s
638:	learn: 0.1187079	total: 1m 41s	remaining: 57.1s
639:	learn: 0.1186328	total: 1m 41s	remaining: 56.9s
640:	learn: 0.1184797	total: 1m 41s	remaining: 56.8s
641:	learn: 0.1183857	total: 1m 41s	remaining: 56.6s
642:	learn: 0.1183421	total: 1m 41s	remaining: 56.4s
643:	learn: 0.1182793	total: 1m 41s	remaining: 56.

781:	learn: 0.1100251	total: 2m 3s	remaining: 34.5s
782:	learn: 0.1100235	total: 2m 4s	remaining: 34.4s
783:	learn: 0.1100234	total: 2m 4s	remaining: 34.2s
784:	learn: 0.1099793	total: 2m 4s	remaining: 34.1s
785:	learn: 0.1099078	total: 2m 4s	remaining: 33.9s
786:	learn: 0.1098994	total: 2m 4s	remaining: 33.8s
787:	learn: 0.1098939	total: 2m 5s	remaining: 33.6s
788:	learn: 0.1098885	total: 2m 5s	remaining: 33.5s
789:	learn: 0.1098312	total: 2m 5s	remaining: 33.3s
790:	learn: 0.1097788	total: 2m 5s	remaining: 33.2s
791:	learn: 0.1097655	total: 2m 5s	remaining: 33s
792:	learn: 0.1097426	total: 2m 5s	remaining: 32.8s
793:	learn: 0.1096338	total: 2m 5s	remaining: 32.7s
794:	learn: 0.1095833	total: 2m 6s	remaining: 32.5s
795:	learn: 0.1095405	total: 2m 6s	remaining: 32.4s
796:	learn: 0.1095124	total: 2m 6s	remaining: 32.2s
797:	learn: 0.1094570	total: 2m 6s	remaining: 32s
798:	learn: 0.1094570	total: 2m 6s	remaining: 31.9s
799:	learn: 0.1094385	total: 2m 6s	remaining: 31.7s
800:	learn: 0.10

938:	learn: 0.1033122	total: 2m 27s	remaining: 9.61s
939:	learn: 0.1032837	total: 2m 28s	remaining: 9.45s
940:	learn: 0.1032113	total: 2m 28s	remaining: 9.29s
941:	learn: 0.1031710	total: 2m 28s	remaining: 9.13s
942:	learn: 0.1031269	total: 2m 28s	remaining: 8.98s
943:	learn: 0.1030661	total: 2m 28s	remaining: 8.82s
944:	learn: 0.1030316	total: 2m 28s	remaining: 8.66s
945:	learn: 0.1029606	total: 2m 28s	remaining: 8.5s
946:	learn: 0.1029365	total: 2m 29s	remaining: 8.35s
947:	learn: 0.1029096	total: 2m 29s	remaining: 8.19s
948:	learn: 0.1028919	total: 2m 29s	remaining: 8.03s
949:	learn: 0.1027920	total: 2m 29s	remaining: 7.87s
950:	learn: 0.1027079	total: 2m 29s	remaining: 7.71s
951:	learn: 0.1026678	total: 2m 29s	remaining: 7.56s
952:	learn: 0.1026507	total: 2m 30s	remaining: 7.4s
953:	learn: 0.1026426	total: 2m 30s	remaining: 7.24s
954:	learn: 0.1026375	total: 2m 30s	remaining: 7.08s
955:	learn: 0.1026300	total: 2m 30s	remaining: 6.93s
956:	learn: 0.1026271	total: 2m 30s	remaining: 6

CatBoostError: (No such file or directory) util/system/file.cpp:848: can't open "../saved/" with mode WrOnly|CreateAlways|Seq (0x00000034)

In [281]:
model.best_score_

{}

In [282]:
features = model.model.get_feature_importance()

CatBoostError: Model has no meta information needed to calculate feature importances.                         Pass training dataset to this function.

In [230]:
the_most_important = features.argsort()[-3:][::-1]

In [233]:
np.array(charset)[the_most_important]

array(['.', 'o', 'm'], dtype='<U1')

In [235]:
user.describe()

,y,len,dots,digits,dash
count,203995.0,203995.000000,203995.000000,203995.000000,203995.000000
mean,1.0,16.231648,1.266619,0.108836,0.123910
std,0.0,5.180874,0.518640,0.685331,0.415591
min,1.0,2.000000,0.000000,0.000000,0.000000
25%,1.0,13.000000,1.000000,0.000000,0.000000
50%,1.0,16.000000,1.000000,0.000000,0.000000
75%,1.0,19.000000,1.000000,0.000000,0.000000
max,1.0,67.000000,6.000000,36.000000,10.000000
